In [2]:
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from functools import partial

In [3]:
df = pd.read_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/train.csv')
corr = pd.read_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/train_updates_20220929.csv')
corr_ids = list(corr['seq_id'])
df = df[~df['seq_id'].isin(corr_ids)]

In [5]:
wild_type = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'
print(df.head())
letters = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

   seq_id                                   protein_sequence   pH  \
0       0  AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...  7.0   
1       1  AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...  7.0   
2       2  AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...  7.0   
3       3  AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...  7.0   
4       4  AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...  7.0   

                         data_source    tm  
0  doi.org/10.1038/s41592-020-0801-4  75.7  
1  doi.org/10.1038/s41592-020-0801-4  50.5  
2  doi.org/10.1038/s41592-020-0801-4  40.5  
3  doi.org/10.1038/s41592-020-0801-4  47.2  
4  doi.org/10.1038/s41592-020-0801-4  49.5  


In [7]:
def similar(s, c, w, b, side=None): 
    if side == 'start':
        return SequenceMatcher(None, s[c][:b], w).ratio()
    elif side == 'end':
        return SequenceMatcher(None, s[c][-b:], w).ratio()
    elif side == None:
        return SequenceMatcher(None, s[c][:], w).ratio()
    else:
        raise ValueError('side must be `start`, `end`, or None')
        
def match_columns(wild, train):
    bounds = [1, 3, 5, 15, 30]
    for bound in bounds:
        start_wild = wild[:bound]
        end_wild = wild[-bound:]
        train[f'first{bound}'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound, side='start'), axis=1)
        train[f'last{bound}'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound, side='end'), axis=1)
    train['sim_ratio'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound), axis=1)
    return train

col = wild_type
length = len(col)
tenths = [int((length/10)*(x+1)) for x in range(10)]
wild_list = []
for idx, ten in enumerate(tenths):
    col_tenth = col[idx:ten]
    col_dict = {}
    for letter in col_tenth:
        if letter in col_dict:
            col_dict[letter] += 1
        else:
            col_dict[letter] = 1
    for letter in letters:
        if letter not in col_dict:
            col_dict[letter] = 0
    for key, value in col_dict.items():
        col_dict[key] = value/(length/10)
    wild_list.append(col_dict)


def tenths(train, letters):
    col = train['protein_sequence']
    length = len(col)
    tenths = [int((length/10)*(x+1)) for x in range(10)]
    tenth_list = []
    for idx, ten in enumerate(tenths):
        col_tenth = col[idx:ten]
        col_dict = {}
        for letter in col_tenth:
            if letter in col_dict:
                col_dict[letter] += 1
            else:
                col_dict[letter] = 1
        for letter in letters:
            if letter not in col_dict:
                col_dict[letter] = 0
        for key, value in col_dict.items():
            col_dict[key] = value/(length/10)
        tenth_list.append(col_dict)
    labels = np.arange(0, 110, 10)
    for idx, (t, w) in enumerate(zip(tenth_list, wild_list)):
        for l in letters:
            train[f'{labels[idx]}-{labels[idx+1]}{l}_count'] = t[l] - w[l]
    return train
        
    
        
        

In [8]:
df = df.apply(partial(tenths, letters=letters), axis=1)

In [10]:
print(df.head(5))

   seq_id                                   protein_sequence   pH  \
0       0  AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...  7.0   
1       1  AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...  7.0   
2       2  AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...  7.0   
3       3  AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...  7.0   
4       4  AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...  7.0   

                         data_source    tm  0-10A_count  0-10R_count  \
0  doi.org/10.1038/s41592-020-0801-4  75.7     0.202757     0.058651   
1  doi.org/10.1038/s41592-020-0801-4  50.5     0.084327     0.069930   
2  doi.org/10.1038/s41592-020-0801-4  40.5     0.090589     0.120724   
3  doi.org/10.1038/s41592-020-0801-4  47.2     0.135917     0.113208   
4  doi.org/10.1038/s41592-020-0801-4  49.5    -0.028472     0.055134   

   0-10N_count  0-10D_count  0-10C_count  ...  90-100L_count  90-100K_count  \
0    -0.090498    -0.015923     0.000000  ...       0.557981      -0.6460

In [248]:
df = match_columns(wild_type, df)

In [188]:
df['seq_len'] = df['protein_sequence'].str.len()
df['seq_len'] = df['seq_len']/df['seq_len'].max()

In [206]:
df['rel_stab'] = df['tm']/df['tm'].max()


/var/folders/7v/_90wc0ds3jd2h03mn95j4sfc0000gn/T/ipykernel_16132/2367815713.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['seq_list'] = [x for x in df['protein_sequence'].str]


ValueError: Length of values (8798) does not match length of index (28956)

In [ ]:
print(df['seq_list'])